In [122]:
from pathlib import Path
import os
ROOT = Path.cwd()
DATA = "../data"
QUERY_DIR = "queries/02-chinook_practice_sqls/"
RESULT_DIR = "results/02-chinook_practice_sqls_results/"

db_path = "../data/Chinook_Sqlite.sqlite"

In [123]:
import sqlite3, pandas as pd

def run_sql(sql: str, params=None):
    with sqlite3.connect(db_path) as con:
        return pd.read_sql_query(sql, con, params=params)

def run_sql_file_to_csv(sql_file, out_csv):
    sql_text = Path(sql_file).read_text(encoding="utf-8")
    df = run_sql(sql_text)
    Path(out_csv).parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(out_csv, index=False)
    return df


run_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

,name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


In [176]:
run_sql("""
           SELECT * FROM genre
        """ )

,GenreId,Name
0,1,Rock
1,2,Jazz
2,3,Metal
3,4,Alternative & Punk
4,5,Rock And Roll
5,6,Blues
6,7,Latin
7,8,Reggae
8,9,Pop
9,10,Soundtrack


In [189]:
run_sql("""
            WITH genre_rev AS (
            SELECT g.GenreId AS Id, g.Name AS Genre, 
            SUM(i.Total) AS revenue
            FROM Invoice i
            FULL JOIN InvoiceLine il  ON il.InvoiceId = i.InvoiceId
            FULL JOIN Track t         ON t.TrackId = il.TrackId
            FULL JOIN Genre g         ON g.GenreId = t.GenreId
            GROUP BY g.GenreId
            )
            SELECT *, ROUND(100 * revenue / (SELECT SUM(revenue) FROM genre_rev), 2) AS percentage
            FROM genre_rev
        """ )

,Id,Genre,revenue,percentage
0,1,Rock,7720.02,37.03
1,2,Jazz,746.46,3.58
2,3,Metal,2093.13,10.04
3,4,Alternative & Punk,1961.66,9.41
4,5,Rock And Roll,83.16,0.40
5,6,Blues,429.66,2.06
6,7,Latin,3472.55,16.66
7,8,Reggae,332.64,1.60
8,9,Pop,239.75,1.15
9,10,Soundtrack,242.55,1.16


In [194]:
queries = [
                    "q01_top_artists.sql", 
                    "q02_rev_by_country.sql", 
                    "q03_monthly_revenue.sql", 
                    "q04_customer_total_spendings_and_avg_order_amount.sql",
                    "q05_10_best_selling_tracks.sql",
                    "q06_number_of_customers_and_revenue_by_support_representative.sql",
                    "q07_unique_customers_who_purchased_rock_genre.sql",
                    "q08_revenue_and_percentage_of_total_by_genre.sql"
                                                                            ]

for sql_name in queries:
    sql_file = os.path.join(QUERY_DIR, sql_name)
    out_csv = os.path.join(RESULT_DIR, (os.path.splitext(sql_name)[0] + ".csv"))
    df = run_sql_file_to_csv(sql_file, out_csv)
    print("Saved:", out_csv, "| rows:", len(df))

Saved: results/02-chinook_practice_sqls_results/q01_top_artists.csv | rows: 5
Saved: results/02-chinook_practice_sqls_results/q02_rev_by_country.csv | rows: 24
Saved: results/02-chinook_practice_sqls_results/q03_monthly_revenue.csv | rows: 60
Saved: results/02-chinook_practice_sqls_results/q04_customer_total_spendings_and_avg_order_amount.csv | rows: 59
Saved: results/02-chinook_practice_sqls_results/q05_10_best_selling_tracks.csv | rows: 10
Saved: results/02-chinook_practice_sqls_results/q06_number_of_customers_and_revenue_by_support_representative.csv | rows: 3
Saved: results/02-chinook_practice_sqls_results/q07_unique_customers_who_purchased_rock_genre.csv | rows: 1
Saved: results/02-chinook_practice_sqls_results/q08_revenue_and_percentage_of_total_by_genre.csv | rows: 25
